In [1]:
%matplotlib inline
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import glob
import os
import matplotlib.pyplot as plt
import pydicom # for reading dicom files at train, test datasets
import tensorflow as tf

In [2]:
# classes
class Patient:
    """class for patient information"""
    def __init__(self, name, imnumber):
        self.name=name
        self.imnumber=imnumber
    
    def makeArray(self, baseDir = 'train'):
        """extract the DICOM image and load it into a numpy array
        """
        #create path to file
        if baseDir == 'test':
            base = tests+'/'+self.name
        else:
            base = trains+'/'+self.name
        pass_dicom = self.imnumber+".dcm"

        #extract file
        filename = pydicom.data.data_manager.get_files(base, pass_dicom)[0]
        ds = pydicom.dcmread(filename)
        #extract the array representing the DICOM image
        return ds.pixel_array
        

In [3]:
# functions
def loadPatient(name, imgNum, baseDir = "train"):
    """create a patient object given a UID and image number
    """
    # choose path based on whether we want to look in the training or testing data
    if baseDir == "test":
        path = tests
    else:
        path = trains
    # extract the folder specific to the name to get image numbers
    path = path + '/{}'.format(name)
    img_numbers = [s.split('/')[-1][:-4] for s in 
                   glob.glob(path+'/*.dcm')]
    # create a patient object if the imgNum is in the set of image numbers
    if imgNum in img_numbers:
        return Patient(name, imgNum)
    
def idVertebrae(name):
    '''takes an index and returns the affected vertebrae
    '''
    # locate record under the name
    inst = training[training['StudyInstanceUID']==name]
    # get list of vertebrae
    vertebrae = list(training.columns[2:])
    # map vertebrae names to 0-1 values
    df = inst[vertebrae]
    df = pd.DataFrame(np.where(df == 1, df.columns, np.nan), columns=df.columns)
    # create list naming affected vertebrae
    return df.loc[0, :].dropna().values.flatten().tolist()

def synthesizeArray(name, number, baseDir = 'train'):
    '''synthesize a numpy array for a given UID
    '''
    # create patient object
    image = loadPatient(name, str(number))
    # build an array of the images
    try:
        return image.makeArray(baseDir)
        
    except:
        return np.nan
    
def pictureBreak(name, num, x,y,w,h, baseDir = 'train'):
    """produce an image of the fracture in the bone
    """
    try:
        Img = synthesizeArray(name,num, baseDir)
        a,b,c,d = int(x), int(w+x), int(y), int(y+h)
        return Img[a:b,c:d]
    except:
        return np.nan
    
def retrieve_slices(name, k, baseDir = 'train'):
    '''return the first k slice numbers for a given patient UID
    '''
    if baseDir == 'test':
        h = glob.glob(tests+'/'+name+'/*.dcm')
    else:
        h = glob.glob(trains+'/'+name+'/*.dcm')
    b = [int(s.split('/')[-1][:-4]) for s in h]
    return b[:k]

def make_label(name):
    '''label UID as healthy or fractured
    '''
    myDict = dict(zip(training['StudyInstanceUID'], training['patient_overall']))
    if myDict[name] == 0:
        return myDict[name], 'healthy :-)'
    else:
        return myDict[name], '-'.join(idVertebrae(name))
    
def normalize_img(data, label):
    """normalize the images in the training data
    """
    tensor = tf.cast(data, dtype=tf.float32)
    tensor = tf.math.divide_no_nan(
            tf.subtract(tensor, tf.reduce_min(tensor)), 
            tf.reduce_max(tensor)
        )
    return tensor, label 

def prediction(img_array):
    """predict presence of break using the model
    """
    #predict probability of vertebrae break using model
    predictions = model.predict(img_array)
    #convert prediction to probability
    score = tf.nn.softmax(predictions[0])
    #provide score and result
    return np.max(score), np.argmax(score)

def make_coords():
    """generate coordinates
    """
    a = np.random.randint(min(bounds['x']),max(bounds['x']))
    b = np.random.randint(min(bounds['y']),max(bounds['y']))
    c = np.random.randint(min(bounds['width']),max(bounds['width']))
    d = np.random.randint(min(bounds['height']),max(bounds['height']))
    return a,b,c,d

def testingImages(name, baseDir = 'train', k=100):
    '''retrieve and format testing data images for prediction
    '''
    pictures = []
    # pick out first 100 slices
    d = retrieve_slices(name, k, baseDir)
    for sl in d:
        # generate coordinates of the break location
        x,y,width,height = make_coords()
        # image of break
        j = pictureBreak(name, sl, x,y,width,height, baseDir)
        # return resized if possible
        try:
            pictures.append(st.resize(j, (64,64,1)))
        except:
            continue
    return pictures

def get_best(testimgs):
    '''get the best prediction from the model
    '''
    preDict = {}
    for img in testimgs:
        try:
            img = np.array([img])
            a,b = prediction(img)
            preDict[a] = b
        except:
            continue

    #m = max(preDict.keys())
    return preDict

def get_label(my_dict):
    '''use the model to make predictions
    '''
    # run the model on images in named directory
    #my_dict = get_best(testingImages(name, baseDir, k))
    #invert the dictionary to provide label names and average probability
    my_inverted_dict = {}
    for k,v in my_dict.items():
        my_inverted_dict.setdefault(translator[v], list()).append(k)
    # smooth results
    stats = {k:np.mean(my_inverted_dict[k]) for k in my_inverted_dict.keys()}
    # only state that the bone is healthy if there are no predictions indicating fractures
    if list(stats.keys()) == ['0, healthy :-)']:
        f = list(stats.keys())[0]
        return f, stats[f]
    else:
        del stats['0, healthy :-)']
        f = max(stats, key=stats.get)
        return f, stats[f]
    
def predict_new(name):
    '''use model to predict on new data outside the training set'''
    C = retrieve_slices(name,10, baseDir="test")
    bob = []
    for v in C:
        
        try:
            a,b,c,d = make_coords()
            x = Patient(name,str(v)).makeArray(baseDir="test")
            y= x[a:a+c, b:b+d]
            bob.append(st.resize(y, (64,64,1)))
        except:
            continue
    try:
        return get_label(get_best(np.array(bob)))
    except:
        return 0

def predict_affected(name, affected_vertebrae):
    '''predict if a case is broken and if the stated vertebrae is affected
    '''
    vert = ['C1','C2','C3','C4','C5','C6','C7']
    try:
        # extract probability and label
        b, p = predict_new(name)
        b1 = int(b.split(', ')[0])
        b2 = b.split(', ')[1].split('-')
        if affected_vertebrae in vert:
            # check if the vertebrae is affected by the break
            if affected_vertebrae in b2:
                return p
            else:
                return 0
        else:
            # return the overall probability
            if b2 == ['healthy :-)']:
                return 0
            else:
                return p
    except:
        return 0.0

In [4]:
#load csv files
bounds = pd.read_csv('../input/rsna-2022-cervical-spine-fracture-detection/train_bounding_boxes.csv')
training = pd.read_csv('../input/rsna-2022-cervical-spine-fracture-detection/train.csv')
testing = pd.read_csv('../input/rsna-2022-cervical-spine-fracture-detection/test.csv')

#paths to train/test images
trains = '../input/rsna-2022-cervical-spine-fracture-detection/train_images'
tests = '../input/rsna-2022-cervical-spine-fracture-detection/test_images'

In [5]:
# get patients whose data appears in the list of those with bounding boxes
bbox_p = list(set(bounds['StudyInstanceUID']))
# get patients whose data appears in segmentation
seglist = glob.glob('../input/rsna-2022-cervical-spine-fracture-detection/segmentations/*')
seg_p = [s.split('/')[-1][:-4] for s in seglist]

In [6]:
# get names and slices of healthy vertebrae
segs = training[training['StudyInstanceUID'].isin(seg_p)]
hv = segs[segs['patient_overall']==0]['StudyInstanceUID'].to_list()
# map the names to the slices
name2slice = {hv[i]:sorted(retrieve_slices(hv[i], 150)) for i in range(len(hv))}
# create the dataframe for healthy patients
names = []
slicenums = []
for k in name2slice:
    for v in name2slice[k]:
        names.append(k)
        slicenums.append(v)
healthy = pd.DataFrame({'StudyInstanceUID':names, 
                        'x': bounds['x'].to_list()[:len(names)],
                        'y': bounds['y'].to_list()[:len(names)], 
                        'width': bounds['width'].to_list()[:len(names)], 
                        'height': bounds['height'].to_list()[:len(names)], 
                        'slice_number':slicenums})

In [7]:
#draw samples of the control and treatment groups
A, B = healthy.sample(frac=0.45), bounds.sample(frac=0.45)
#combine the resulting dataframes
df = pd.concat([A, B], axis=0).dropna()

In [8]:
#create arrays of images and labels
a = df.apply(lambda x: pictureBreak(x['StudyInstanceUID'],x["slice_number"], x["x"], x["y"], x["width"], x["height"]),axis=1)
b = df['StudyInstanceUID'].apply(make_label)
# assemble into data frame with NaN elements removed
myDict = {'img_array': a.tolist(), 'label_array': b.tolist()}
df = pd.DataFrame(myDict).dropna()

In [9]:
import skimage.transform as st
#prepare feature data for load
x_train = np.array(df['img_array'])
x_train = np.array([st.resize(val,(64,64)) for val in x_train])
#prepare target data for load
y_train = df["label_array"].tolist()
y_train = [str(y[0])+", "+y[1] for y in y_train]

In [10]:
unquies = list(set(y_train))
numDict={unquies[i]:i for i in range(len(unquies))}
translator = {v:k for k,v in numDict.items()}
numz = [numDict[y] for y in y_train]
y_train=numz

In [11]:
# data augmentation layer
data_augmentation = tf.keras.Sequential([
  tf.keras.layers.RandomFlip("horizontal_and_vertical"),
  tf.keras.layers.RandomRotation(0.3),
  tf.keras.layers.RandomZoom(0.1)
])

def acceptor(img, label):
    '''function to format the data for augmentation
    '''
    image = tf.cast(tf.expand_dims(img, -1), tf.float32)
    return image, label

In [12]:
#create tensorflow data set (the fun part)
my_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train))
#shuffle the data
my_ds = my_ds.shuffle(len(df.index))
#normalize the image so that everything is between 0 and 1
my_ds = my_ds.map(normalize_img, num_parallel_calls=tf.data.AUTOTUNE)
# augment the data
my_ds = my_ds.map(acceptor, num_parallel_calls=tf.data.AUTOTUNE)
my_ds = my_ds.map(lambda x, y: (data_augmentation(x), y),  num_parallel_calls=4)

In [13]:
#build a simple model
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(16, 3, padding='same', activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(32, 3, padding='same', activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(64, 3, padding='same', activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Dropout(0.15),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(max(numDict.values())+1)
])
#compile the model
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [14]:
#separate data set into validation and training sets
valid_ds_f = my_ds.take(int(0.2*4432))
train_ds_f = my_ds.skip(int(0.2*4432))
#batch datasets
valid_ds = valid_ds_f.batch(32)
train_ds = train_ds_f.batch(32)

In [15]:
#set the number of epochs
epochs=100
#fit the model
output = model.fit(train_ds,
          validation_data=valid_ds,
          epochs=epochs)

Epoch 1/100
111/111 [==============================] - 7s 56ms/step - loss: 2.4036 - accuracy: 0.5320 - val_loss: 2.2828 - val_accuracy: 0.5327
Epoch 2/100
111/111 [==============================] - 6s 54ms/step - loss: 2.2564 - accuracy: 0.5394 - val_loss: 2.2525 - val_accuracy: 0.5293
Epoch 3/100
111/111 [==============================] - 6s 55ms/step - loss: 2.2368 - accuracy: 0.5343 - val_loss: 2.2083 - val_accuracy: 0.5451
Epoch 4/100
111/111 [==============================] - 6s 55ms/step - loss: 2.2405 - accuracy: 0.5238 - val_loss: 2.1617 - val_accuracy: 0.5429
Epoch 5/100
111/111 [==============================] - 7s 61ms/step - loss: 2.1809 - accuracy: 0.5295 - val_loss: 2.1677 - val_accuracy: 0.5384
Epoch 6/100
111/111 [==============================] - 6s 53ms/step - loss: 2.1664 - accuracy: 0.5238 - val_loss: 2.0790 - val_accuracy: 0.5429
Epoch 7/100
111/111 [==============================] - 6s 54ms/step - loss: 2.1148 - accuracy: 0.5289 - val_loss: 2.0532 - val_accuracy:

In [16]:
# extract entries in the testing data
M = glob.glob(tests+"/*")
N = [M[i].split('/')[-1] for i in range(len(M))]

In [17]:
# Fix mismatch with test_images folder
testing = pd.DataFrame(columns = ['row_id','StudyInstanceUID','prediction_type'])
for i in N:
    for j in ['C1','C2','C3','C4','C5','C6','C7','patient_overall']:
        testing = testing.append({'row_id':i+'_'+j,'StudyInstanceUID':i,'prediction_type':j},ignore_index=True)

testing['fractured'] = testing.apply(lambda x: predict_affected(x['StudyInstanceUID'], x['prediction_type']),axis=1)
testing = testing.dropna()

In [18]:
sub = testing[['row_id', 'fractured']]
sub.to_csv("submission.csv", index=False)